# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [2]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to neil.maude@gmail.com and will expire on January 29, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1184 - Server binary: C:\Users\Demo\Anaconda2\envs\dato-env\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Demo\AppData\Local\Temp\graphlab_server_1456574048.log.0
[INFO] GraphLab Server Version: 1.8.1


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 2.704343     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 2.875050     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [7]:
model_all.get("coefficients").print_rows(num_rows=100)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

###Quiz solution
Bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [58]:
import numpy as np # note this allows us to refer to numpy as np instead 

print 'Set of L1_penalty values :',np.logspace(1,7, num=13)
print 'Set of features :', all_features

min_RSS = 0
l1_pen_best = 0
predictions = []
sq_error = []
for l1_pen in np.logspace(1,7,num=13):
    # fit the model
    print 'Computing for L1_penalty = ', l1_pen
    model_temp = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pen,
                                              verbose=False)
    # compute RSS on the validation set
    predictions = model_temp.predict(validation)
    errors = graphlab.SFrame(predictions)
    errors['error'] = errors['X1'] - validation['price']
    errors['prediction'] = predictions
    errors['data'] = validation['price']
    errors['sq'] = errors['error'].apply(lambda x: x**2)
    #print 'Errors: ', errors.head()
    sq_error = errors['error'].apply(lambda x: x**2)
    model_RSS = errors['sq'].sum()   #sq_error.sum()
    if min_RSS == 0:
        min_RSS = model_RSS
        l1_pen_best = l1_pen
        print 'First L1_penalty/RRS :',l1_pen, model_RSS
    else:
        if model_RSS < min_RSS:
            min_RSS = model_RSS
            l1_pen_best = l1_pen
            print 'Better L1_penalty/RRS found :',l1_pen, model_RSS 
        else:
            print 'No improvement at L1_penalty : ', l1_pen, model_RSS
            
print 'Best L1_penalty : ', l1_pen_best, ' RSS = ', min_RSS

Set of L1_penalty values : [  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]
Set of features : ['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
Computing for L1_penalty =  10.0
First L1_penalty/RRS : 10.0 6.25766285142e+14
Computing for L1_penalty =  31.6227766017
No improvement at L1_penalty :  31.6227766017 6.25766285362e+14
Computing for L1_penalty =  100.0
No improvement at L1_penalty :  100.0 6.25766286058e+14
Computing for L1_penalty =  316.227766017
No improvement at L1_penalty :  316.227766017 6.25766288257e+14
Computing for L1_penalty =  1000.0
No improvement at L1_penalty :  1000.0 6.25766295212e+14
Computing for L1_

In [46]:
pred = [1,2]
values = [0.5,1]
sf = (graphlab.SFrame(pred)['X1']) - (graphlab.SFrame(values)['X1'])
print sf
#print pred.apply(lambda x: x**2)
print sum(pred)
sf = graphlab.SFrame(pred)
print sf['X1'].apply(lambda x: x**2).sum()
print sf
print sf['X1'].sum()
print 'Rows: ',len(training)
#model_all.predict(training)
training['price']

[0.5, 1.0]
3
5
+----+
| X1 |
+----+
| 1  |
| 2  |
+----+
[2 rows x 1 columns]

3
Rows:  9761


dtype: float
Rows: 9761
[604000.0, 1225000.0, 468000.0, 310000.0, 400000.0, 650000.0, 485000.0, 385000.0, 285000.0, 252700.0, 329000.0, 233000.0, 937000.0, 438000.0, 322500.0, 605000.0, 861990.0, 309000.0, 488000.0, 210490.0, 785000.0, 1350000.0, 228000.0, 345000.0, 600000.0, 650000.0, 289000.0, 975000.0, 204000.0, 1325000.0, 1040000.0, 571000.0, 360000.0, 380000.0, 480000.0, 410000.0, 720000.0, 315000.0, 940000.0, 305000.0, 215000.0, 243500.0, 1099880.0, 430000.0, 700000.0, 199000.0, 314000.0, 437500.0, 850830.0, 290000.0, 460000.0, 680000.0, 272500.0, 518500.0, 171800.0, 391500.0, 1450000.0, 260000.0, 430000.0, 212000.0, 453246.0, 220000.0, 452000.0, 665000.0, 420000.0, 500000.0, 258000.0, 511000.0, 532170.0, 282950.0, 320000.0, 255000.0, 420000.0, 370000.0, 269950.0, 653000.0, 425000.0, 470000.0, 163500.0, 560000.0, 585188.0, 400000.0, 230000.0, 423000.0, 465000.0, 440000.0, 297000.0, 470000.0, 226500.0, 677900.0, 425000.0, 180250.0, 464000.0, 320000.0, 625504.0, 467000.0, 686000.0,

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

Best L1_penalty :  10  RSS =  6.258e+14

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [54]:
model_best_l1 = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=10,
                                              verbose=False)
model_best_l1.get('coefficients').print_rows(num_rows=100)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

###Quiz question
At best L1_penalty of 10,000,000, there are *no* no-zero weights - all features are included...

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [59]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [60]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [61]:
print 'Set of L1_penalty values :', l1_penalty_values
print 'Set of features :', all_features

#min_RSS = 0
#l1_pen_best = 0
#predictions = []
#sq_error = []
for l1_pen in l1_penalty_values:
    # fit the model
    print 'Computing for L1_penalty = ', l1_pen
    model_temp = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pen,
                                              verbose=False)
    n_non_zero = model_temp['coefficients']['value'].nnz()
    print 'l1_penalty = ', l1_pen, ' number of coefficients = ', n_non_zero
    # compute RSS on the validation set
    #predictions = model_temp.predict(validation)
    #sq_error = graphlab.SFrame(predictions)['X1'].apply(lambda x: x**2)
    #model_RSS = sq_error.sum()
    #if min_RSS == 0:
    #    min_RSS = model_RSS
    #    l1_pen_best = l1_pen
    #    print 'First L1_penalty/RRS :',l1_pen, model_RSS
    #else:
    #    if model_RSS < min_RSS:
    #        min_RSS = model_RSS
    #        l1_pen_best = l1_pen
    #        print 'Better L1_penalty/RRS found :',l1_pen, model_RSS 
    #    else:
    #        print 'No improvement at L1_penalty : ', l1_pen, model_RSS
            
#print 'Best L1_penalty : ', l1_pen_best, ' RSS = ', min_RSS

Set of L1_penalty values : [  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]
Set of features : ['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
Computing for L1_penalty =  100000000.0
l1_penalty =  100000000.0  number of coefficients =  18
Computing for L1_penalty =  127427498.57
l1_penalty =  127427498.57  number of coefficients =  18
Computing for L1_penalty =  162377673.919
l1_penalty =  162377673.919  number of coefficients =  18
Computing for L1_penalty =  206913808.111
l1_penalty =  206913808.111  numbe

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [62]:
print 'Set of L1_penalty values :', l1_penalty_values
#print 'Set of features :', all_features
print 'Looking for maximum non-zeros :',max_nonzeros

# l1_pen is always increasing...
f_found = False

for l1_pen in l1_penalty_values:
    # fit the model
    print 'Computing for L1_penalty = ', l1_pen
    model_temp = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pen,
                                              verbose=False)
    n_non_zero = model_temp['coefficients']['value'].nnz()
    print 'l1_penalty = ', l1_pen, ' number of coefficients = ', n_non_zero
    if n_non_zero > max_nonzeros:
        l1_penalty_min = l1_pen     #keep over-writing until condition is no longer true
        print 'Greater than max zeros at l1_penalty = ', l1_pen
    else:
        if f_found==False:
            l1_penalty_max = l1_pen #first time this happens
            f_found = True
            print 'Found max zeros at l1_penalty = ', l1_pen
            
print 'Solution minimum=', l1_penalty_min, ' maximum=',l1_penalty_max
#l1_penalty_min = 
#l1_penalty_max = 

Set of L1_penalty values : [  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]
Looking for maximum non-zeros : 7
Computing for L1_penalty =  100000000.0
l1_penalty =  100000000.0  number of coefficients =  18
Greater than max zeros at l1_penalty =  100000000.0
Computing for L1_penalty =  127427498.57
l1_penalty =  127427498.57  number of coefficients =  18
Greater than max zeros at l1_penalty =  127427498.57
Computing for L1_penalty =  162377673.919
l1_penalty =  162377673.919  number of coefficients =  18
Greater than max zeros at l1_penalty =  162377673.919
Computing for L1_penalty =  206913808.111
l1_penalty =  206913808.111  number of coefficients =  18
Greater than max zeros at l1_penalty =  

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

### Solution:
Solution minimum= 2976351441.63  maximum= 3792690190.73

This means that 2976351441 is the minimum value at which the solution will have more zeros (l1_penalty_min)

But 3792690190 is the smallest test penalty that has fewer zeros (l1_penalty_max)

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [63]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [64]:
print 'Set of L1_penalty values :',l1_penalty_values
#print 'Set of features :', all_features

min_RSS = 0
l1_pen_best = 0
predictions = []
sq_error = []
for l1_pen in l1_penalty_values:
    # fit the model
    print 'Computing for L1_penalty = ', l1_pen
    model_temp = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_pen,
                                              verbose=False)
    n_non_zero = model_temp['coefficients']['value'].nnz()
    # compute RSS on the validation set
    #predictions = model_temp.predict(validation)
    #sq_error = graphlab.SFrame(predictions)['X1'].apply(lambda x: x**2)
    #model_RSS = sq_error.sum()
    predictions = model_temp.predict(validation)
    errors = graphlab.SFrame(predictions)
    errors['error'] = errors['X1'] - validation['price']
    errors['prediction'] = predictions
    errors['data'] = validation['price']
    errors['sq'] = errors['error'].apply(lambda x: x**2)
    #print 'Errors: ', errors.head()
    sq_error = errors['error'].apply(lambda x: x**2)
    model_RSS = errors['sq'].sum()   #sq_error.sum()
    print 'l1_penalty: ', l1_pen, ', non-zeros :', n_non_zero, ', RSS = ', model_RSS
    if n_non_zero == max_nonzeros:
        if min_RSS == 0:
            min_RSS = model_RSS
            l1_pen_best = l1_pen
            print 'First L1_penalty/RRS :',l1_pen, model_RSS
        else:
            if model_RSS < min_RSS:
                min_RSS = model_RSS
                l1_pen_best = l1_pen
                print 'Better L1_penalty/RRS found :',l1_pen, model_RSS 
            else:
                print 'No improvement at L1_penalty : ', l1_pen, model_RSS
            
print 'Best L1_penalty : ', l1_pen_best, ' RSS = ', min_RSS

Set of L1_penalty values : [  2.97635144e+09   3.01931664e+09   3.06228184e+09   3.10524703e+09
   3.14821223e+09   3.19117743e+09   3.23414263e+09   3.27710782e+09
   3.32007302e+09   3.36303822e+09   3.40600341e+09   3.44896861e+09
   3.49193381e+09   3.53489901e+09   3.57786420e+09   3.62082940e+09
   3.66379460e+09   3.70675980e+09   3.74972499e+09   3.79269019e+09]
Computing for L1_penalty =  2976351441.63
l1_penalty:  2976351441.63 , non-zeros : 10 , RSS =  9.66925692362e+14
Computing for L1_penalty =  3019316638.95
l1_penalty:  3019316638.95 , non-zeros : 10 , RSS =  9.74019450085e+14
Computing for L1_penalty =  3062281836.27
l1_penalty:  3062281836.27 , non-zeros : 10 , RSS =  9.81188367942e+14
Computing for L1_penalty =  3105247033.59
l1_penalty:  3105247033.59 , non-zeros : 10 , RSS =  9.89328342459e+14
Computing for L1_penalty =  3148212230.92
l1_penalty:  3148212230.92 , non-zeros : 10 , RSS =  9.98783211266e+14
Computing for L1_penalty =  3191177428.24
l1_penalty:  3191177

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

### Question 1:
Best L1_penalty :  3448968612.16  RSS =  1.04693748875e+15


In [65]:
model_temp = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=3448968612.16  ,
                                              verbose=False)
model_temp.get('coefficients').print_rows(num_rows=100)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |

### Question 2:
Features:

bedrooms, bathrooms, sqft_living, sqft_living_sqrt, grade, sqft_above